In [398]:
# import packages
import numpy as np
import pandas as pd
import psycopg2
from pymongo import MongoClient
import pysolr

### Building PostgreSQL

In [399]:
# import the data
data = pd.read_csv('steam.csv')
data.head()

,appid,name,release_date,english,developer,publisher,platforms,required_age,categories,genres,steamspy_tags,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,owners,price
0,10,Counter-Strike,2000-11-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,124534,3339,17612,317,10000000-20000000,7.19
1,20,Team Fortress Classic,1999-04-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,3318,633,277,62,5000000-10000000,3.99
2,30,Day of Defeat,2003-05-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Valve Anti-Cheat enabled,Action,FPS;World War II;Multiplayer,0,3416,398,187,34,5000000-10000000,3.99
3,40,Deathmatch Classic,2001-06-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,1273,267,258,184,5000000-10000000,3.99
4,50,Half-Life: Opposing Force,1999-11-01,1,Gearbox Software,Valve,windows;mac;linux,0,Single-player;Multi-player;Valve Anti-Cheat en...,Action,FPS;Action;Sci-fi,0,5250,288,624,415,5000000-10000000,3.99


In [400]:
# Establishing connection for postgreSQL
conn = psycopg2.connect(
    host="my-postgres",
    database="postgres",
    user="postgres",
    password="password"
)

In [403]:
# Creating a cursor object
cursor = conn.cursor()

In [405]:
cursor.execute("ROLLBACK")

In [406]:
# Creating a database
sql = '''CREATE DATABASE STEAM'''

cursor.execute(sql)
print("Database created successfully........")

Database created successfully........


In [407]:
# Creating table
sql = '''
CREATE TABLE GAMES(
    ID INT PRIMARY KEY,
    NAME VARCHAR(5000),
    RELEASE_DATE VARCHAR(5000),
    ENGLISH INT,
    DEVELOPER VARCHAR(5000),
    PUBLISHER VARCHAR(5000),
    PLATFORMS VARCHAR(5000),
    REQUIRED_AGE INT,
    CATEGORIES VARCHAR(5000),
    GENRES VARCHAR(5000),
    STEAMSPY_TAGS VARCHAR(5000),
    ACHIEVEMENTS INT,
    POSITIVE_RATINGS INT,
    NEGATIVE_RATINGS INT,
    AVERAGE_PLAYTIME INT,
    MEDIAN_PLAYTIME INT,
    OWNERS VARCHAR(5000),
    PRICE FLOAT
)
'''

cursor.execute(sql)
print("Table created successfully........")

Table created successfully........


In [408]:
# Populate the table with data
data_list = [list(row) for row in data.itertuples(index=False)] 

for i in range(len(data_list)):
    cursor.execute("INSERT INTO games VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)",
                   data_list[i])

### Building MongoDB

In [409]:
# Data Preprocessing
genre_data = pd.read_csv('steam.csv')[['appid', 'genres', 'steamspy_tags']]
genre_data['genres'] = genre_data['genres'].apply(lambda x: x.split(';'))
genre_data['steamspy_tags'] = genre_data['steamspy_tags'].apply(lambda x: x.split(';'))
genre_data.head()

,appid,genres,steamspy_tags
0,10,[Action],"[Action, FPS, Multiplayer]"
1,20,[Action],"[Action, FPS, Multiplayer]"
2,30,[Action],"[FPS, World War II, Multiplayer]"
3,40,[Action],"[Action, FPS, Multiplayer]"
4,50,[Action],"[FPS, Action, Sci-fi]"


In [410]:
# Establishing connection for MongoDB
client = MongoClient("mongodb://my-mongo", 27017, username='admin', password='password')
db = client.admin

In [411]:
# Create a Collection
mycol = db['genres']

In [412]:
# Populate the Collection
for i in range(genre_data.shape[0]):
    data_dict = {'id': int(genre_data.iloc[i][0]), 'genres': genre_data.iloc[i][1], 'tags': genre_data.iloc[i][2]}
    mycol.insert_one(data_dict)

### Building Apache Solr

In [413]:
# Connect to the established Solr database
solr = pysolr.Solr('http://awesome-hw.sdsc.edu:8983/solr/project-stream-desc-data', timeout=10)

### Building the Query Function for Outputting Results of Keyword Queries

In [414]:
# Helper function for postgreSQL query
def postgresql_search(appids=None, name=None, eng=None, dev=None, pos_rate_per=None, p=None):
    postgresql = "SELECT ID, NAME, PRICE, " + \
    "(CAST(POSITIVE_RATINGS AS FLOAT)/(POSITIVE_RATINGS + NEGATIVE_RATINGS)) AS POSTIVE_RATING_PERCENTAGE " + \
    "FROM GAMES"
    
    if appids!=None or name!=None or eng!=None or dev!=None or pos_rate_per!=None or p!=None:
        postgresql_w = " WHERE"
        
        restriction = 0
        
        if appids==[]:
            return []
        
        if appids!=None:
            if restriction != 0:
                postgresql_w += " AND"
            
            id_counts = 0
            id_list = "("
            for i in appids:
                if id_counts != 0:
                    id_list += ", "
                id_list += str(i)
                id_counts += 1
            id_list += ")"
            
            postgresql_w += " ID IN " + id_list
            restriction += 1
        
        if name!=None:
            if restriction != 0:
                postgresql_w += " AND"
            postgresql_w += " NAME = '" + name + "'"
            restriction += 1
            
        if eng!=None:
            if restriction != 0:
                postgresql_w += " AND"
            postgresql_w += " ENGLISH = " + str(eng)
            restriction += 1
        
        if dev!=None:
            if restriction != 0:
                postgresql_w += " AND"
            postgresql_w += " DEVELOPER = '" + dev + "'"
            restriction += 1
        
        if pos_rate_per!=None:
            if restriction != 0:
                postgresql_w += " AND"
            postgresql_w += " (CAST(POSITIVE_RATINGS AS FLOAT)/(POSITIVE_RATINGS + NEGATIVE_RATINGS)) >= " + str(pos_rate_per)
            restriction += 1
        
        if p!=None:
            if restriction != 0:
                postgresql_w += " AND"
            postgresql_w += " PRICE <= " + str(p)
            restriction += 1
            
        postgresql += postgresql_w
    
    cursor.execute(postgresql)
    
    results = cursor.fetchall()
    
    return results

In [415]:
# Helper function for MongoDB query
def mongodb_search(gen=None, tag=None):
    mongodb_query = {}
    
    if gen!=None or tag!=None:
        mongodb_query['$and'] = []
        
        if gen!=None:
            for genre in gen:
                mongodb_query['$and'].append({'genres': genre})
        
        if tag!=None:
            for t in tag:
                mongodb_query['$and'].append({'tags': t})
    
    results = mycol.find(mongodb_query)
    
    result_ids = []
    for x in results:
        result_ids.append(x['id'])
    
    return result_ids

In [416]:
# Helper function for Solr query
def solr_search(des):
    solr_query = ""
    
    restrictions = 0
    for keyword in des:
        
        if restrictions != 0:
            solr_query += ' AND '
        
        solr_query += '(detailed_description: "' + keyword + '"~2)'
        restrictions += 1
    
    results = solr.search(solr_query, rows = 25000)
    
    result_ids = []
    for x in results:
        result_ids.append(int(x['steam_appid']))
    
    return result_ids

In [417]:
# Build the search function for games with key work inputs
def search_game(name=None, eng=None, dev=None, gen=None, tag=None, pos_rate_per=None, p=None, des=None):
    appids = None
    if gen!=None or tag!=None or des!=None:
        if gen!=None or tag!=None:
            mongodb_query_id = mongodb_search(gen=gen, tag=tag)
            mongodb_query_id = list(np.unique(mongodb_query_id))
            
        if des!=None or des == []:
            solr_query_id = solr_search(des=des)
            solr_query_id = list(np.unique(solr_query_id))
        
        if (gen!=None or tag!=None) and des!=None:
            appids = list(set(mongodb_query_id).intersection(solr_query_id))
        elif (gen!=None or tag!=None) and des==None:
            appids = mongodb_query_id
        elif (gen==None and tag==None) and des!=None:
            appids = solr_query_id
            
    results = postgresql_search(appids=appids, name=name, eng=eng, dev=dev, pos_rate_per=pos_rate_per, p=p)

    return results

### Code Demos

In [425]:
# Search for game named 'Counter-Strike'
name = 'Counter-Strike'
search_game(name=name)

[(10, 'Counter-Strike', 7.19, 0.9738881546534452)]

In [426]:
# Search for game with price under 5
price = 5
search_game(p = price)

[(20, 'Team Fortress Classic', 3.99, 0.8397873955960516),
 (30, 'Day of Defeat', 3.99, 0.8956476140534871),
 (40, 'Deathmatch Classic', 3.99, 0.8266233766233766),
 (50, 'Half-Life: Opposing Force', 3.99, 0.9479956663055255),
 (60, 'Ricochet', 3.99, 0.8012783265543288),
 (130, 'Half-Life: Blue Shift', 3.99, 0.900990099009901),
 (280, 'Half-Life: Source', 0.0, 0.7815352697095436),
 (320, 'Half-Life 2: Deathmatch', 3.99, 0.8843837226384604),
 (340, 'Half-Life 2: Lost Coast', 0.0, 0.8500661472879611),
 (360, 'Half-Life Deathmatch: Source', 0.0, 0.7422343324250681),
 (440, 'Team Fortress 2', 0.0, 0.9381067983233773),
 (570, 'Dota 2', 0.0, 0.8587102445738107),
 (630, 'Alien Swarm', 0.0, 0.9487919024814976),
 (730, 'Counter-Strike: Global Offensive', 0.0, 0.8679519627192155),
 (1200, 'Red Orchestra: Ostfront 41-45', 3.99, 0.8750700280112045),
 (1630, 'Disciples II: Rise of the Elves ', 4.99, 0.8067978533094812),
 (1640, "Disciples II: Gallean's Return", 4.99, 0.7632398753894081),
 (1670, 'Iro

In [427]:
# Search for game under the genres 'Action' and 'Strategy'
genre = ['Action', 'Strategy']
search_game(gen = genre)

[(570, 'Dota 2', 0.0, 0.8587102445738107),
 (2720, 'ThreadSpace: Hyperbol', 3.99, 0.775),
 (2820, 'X3: Terran Conflict', 15.99, 0.8572178477690289),
 (4520, 'Full Spectrum Warrior', 7.99, 0.6385542168674698),
 (4530, 'Full Spectrum Warrior: Ten Hammers', 4.99, 0.6666666666666666),
 (4560, 'Company of Heroes - Legacy Edition', 0.0, 0.9371563236449332),
 (4920, 'Natural Selection 2', 6.99, 0.8482269503546099),
 (6080, 'Thrillville®: Off the Rails™', 7.19, 0.8774193548387097),
 (6250, 'Making History: The Calm & the Storm', 3.99, 0.8112874779541446),
 (6400, 'Joint Task Force', 5.79, 0.8695652173913043),
 (8170, 'Battlestations Pacific', 6.99, 0.6547770700636942),
 (8500, 'EVE Online', 0.0, 0.7459861871019823),
 (9340, 'Company of Heroes: Opposing Fronts', 9.99, 0.9175911251980983),
 (11240, 'Space Trader: Merchant Marine', 3.99, 0.4909688013136289),
 (12530, 'Hunting Unlimited™ 2008', 4.99, 0.7631578947368421),
 (15740, "Oddworld: Munch's Oddysee", 3.99, 0.7381756756756757),
 (15750, "Od

In [428]:
# Search for game with words 'adventure' and 'shoot' in the description
description = ['adventure', 'shoot']
search_game(des = description)

[(2610, 'GUN™', 9.99, 0.8948004836759371),
 (3020, 'Call of Juarez™', 8.59, 0.8002018163471241),
 (3540, 'Peggle™ Nights', 4.25, 0.9387755102040817),
 (4500, 'S.T.A.L.K.E.R.: Shadow of Chernobyl', 9.99, 0.9509001636661211),
 (8140, 'Tomb Raider: Underworld', 6.99, 0.7496839443742098),
 (15970, 'Luxor', 5.99, 0.6744186046511628),
 (16710, 'Insecticide Part 1', 0.79, 0.7368421052631579),
 (18470, 'Helldorado', 6.99, 0.6973684210526315),
 (45710, 'Dark Void™', 6.49, 0.48660714285714285),
 (49520, 'Borderlands 2', 19.99, 0.929178233600658),
 (60340, 'Luxor: 5th Passage', 6.99, 0.6610169491525424),
 (110610, 'Alien Zombie Megadeath', 3.99, 0.734375),
 (200900, 'Cave Story+', 10.99, 0.9570093457943926),
 (205230, 'Hell Yeah! Wrath of the Dead Rabbit', 9.99, 0.8620470438652257),
 (207000, 'Alien Spidy', 7.99, 0.6029411764705882),
 (216930, 'Luxor: Amun Rising HD', 6.99, 0.8181818181818182),
 (243000, 'Omikron: The Nomad Soul', 6.99, 0.7069486404833837),
 (249130, 'LEGO® Marvel™ Super Heroes',

In [429]:
# Search with multiple keys
english = 1
developer = 'Choice of Games'
genres = ['RPG', 'Adventure']
tags = ['Indie']
pos_rating_percentage = 0.5
price = 5
description = ['kingdom']
search_game(eng=english, dev=developer, gen=genres, tag=tags, pos_rate_per=pos_rating_percentage, p=price, des=description)

[(492370, 'Affairs of the Court: Choice of Romance', 4.79, 0.7735849056603774),
 (540660, 'Sorcery Is for Saps', 2.89, 0.8333333333333334),
 (558410, 'Cannonfire Concerto', 2.89, 0.8421052631578947),
 (722600, 'The Last Monster Master', 2.89, 0.75),
 (922410, 'DinoKnights', 3.99, 0.875)]